# SVD


From the textbook ["Statistics, Data Mining, and Machine Learning in Astronomy" (2013)](http://www.astroml.org/book_figures/chapter7/fig_svd_visual.html)
![asfda](svd.png)

SVD can factorize an $N\times K$ matrix. The column of U is the *left-singular vector*, and the columns of V are the *right-singular vectors*. The convention used in the book is:

$\Sigma$ is dimention $R \times R$ where $R = \min(N,K)$ so $\Sigma$ is a square matrix $R \times R$, U is $N\times R$ and $V^T$ is $R\times K$. 

Let's do an example. Lets define a matrix to decompose


In [229]:
import numpy as np
matrix = np.array([ [1,2,5,6], [2,3,7,8],[6,7,9,50]],dtype='d')
print(matrix)
print('Shape of matrix: {}'.format(matrix.shape))

[[  1.   2.   5.   6.]
 [  2.   3.   7.   8.]
 [  6.   7.   9.  50.]]
Shape of matrix: (3, 4)


We can decompose using the [numpy SVD](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) function:

In [230]:
U, D, Vtranspose = np.linalg.svd(matrix,full_matrices=False)

The *full_matrices=False* ensures that the convection stated above is followed. Let's look at the shapes of U, D and V tranpose. 

In [231]:
print('Shape of U: {}'.format(U.shape))
print('Shape of V Transpose: {}'.format(Vtranspose.shape))
print('Shape of D: {}'.format(D.shape))

Shape of U: (3, 3)
Shape of V Transpose: (3, 4)
Shape of D: (3,)


So D only return the list of the singular values, so we have to construct the diagonal matrix with *np.diag()*. Let's reconstructe the matrix via multiplication. 

In [232]:
Ddiagonal = np.diag(D)
np.dot(U.dot(Ddiagonal) , Vtranspose)

array([[  1.,   2.,   5.,   6.],
       [  2.,   3.,   7.,   8.],
       [  6.,   7.,   9.,  50.]])

Remember that the dimensions of the matrices depend on R. We can try to reconstuct the matrix choosing less singular values $n < R$. So for instant lets reconstruct using the first two singular values

In [233]:
dim = 2
Unew = U[:,0:dim]
Dnew= D[0:dim]
Vtranspose_new = Vtranspose[0:dim,:]

In [234]:
print('Shape of new U: {}'.format(Unew.shape))
print('Shape of new V Transpose: {}'.format(Vtranspose_new.shape))
print('Shape of new D: {}'.format(Dn.shape))

Shape of new U: (3, 2)
Shape of new V Transpose: (2, 4)
Shape of new D: (2,)


Lets reconstruct the matrix

In [235]:
Ddiagonalnew = np.diag(Dnew)
np.dot(Unew.dot(Ddiagonalnew) , Vtranspose_new)

array([[  1.29139716,   2.0855279 ,   4.92872996,   5.96590467],
       [  1.79958314,   2.9411757 ,   7.04901804,   8.02345005],
       [  5.99739176,   6.99923445,   9.00063793,  50.00030518]])

We see it is not exact put a good approximation since the highest singular values account for much of the variation. 

# SVD with Scikit-learn

The same can be achieved using [Truncated SVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) from the scikitlearn package.


We defined the dimeinsion and for the algoritm we use arpack to make scikit-learn call the scipy solver for SVD. 

In [236]:
from sklearn.decomposition import TruncatedSVD
sklearn_svd = TruncatedSVD(n_components=2,algorithm='arpack')
svdfit = sklearn_svd.fit(matrix)

In [237]:
singularvalues = svdfit.singular_values_
print('First two singular values the matrix: {}'.format(singularvalues))

First two singular values the matrix: [ 53.01701453   6.85938137]


We see that the fit model has a lot of attributes. For example the .components gives you the V transpose matrix up to a sign.

> SVD suffers from a problem called “sign indeterminancy”, which means the sign of the components_ and the output from transform depend on the algorithm and random state. To work around this, fit instances of this class to data once, then keep the instance around to do transformations.


One also useful information is the *explained_variance_ratio_ *. This represents the **percentage of variance explained by each of the selected components.**


In [238]:
print(svdfit.components_)
V[0:2]

[[ 0.11968113  0.14407797  0.20238613  0.96122725]
 [ 0.11424978  0.27818971  0.92043332 -0.24971981]]


array([[-0.11968113, -0.14407797, -0.20238613, -0.96122725],
       [-0.11424978, -0.27818971, -0.92043332,  0.24971981]])

In [239]:
print(svdfit.explained_variance_ratio_)

[ 0.97846786  0.02142035]


So the first component explain much of the variance of the data. 

There is also the mathod transform. This actually returns $U \times D$:

In [240]:
matrix_svd = svdfit.transform(matrix)
matrix_svd

array([[  7.1871312 ,   3.77447691],
       [  9.77811705,   5.50834341],
       [ 51.60947008,  -1.56926408]])

In [241]:
Unew.dot(Ddiagonalnew)

array([[ -7.1871312 ,  -3.77447691],
       [ -9.77811705,  -5.50834341],
       [-51.60947008,   1.56926408]])

Where again the columns of U are called the **left-singular vectors**. In theory we can now plot this two as the first two principal components.

We multiply times V transpose and restore the approximation of the matrix:

In [242]:
np.dot(matrix_svd , svdfit.components_)

array([[  1.29139716,   2.0855279 ,   4.92872996,   5.96590467],
       [  1.79958314,   2.9411757 ,   7.04901804,   8.02345005],
       [  5.99739176,   6.99923445,   9.00063793,  50.00030518]])

In [243]:
matrix

array([[  1.,   2.,   5.,   6.],
       [  2.,   3.,   7.,   8.],
       [  6.,   7.,   9.,  50.]])

# Principal Value Decomposition and PCA

Let's see how this two method compare to each other. In summary:

1. The *right-singular vectors* V correspond to the principal components R.
2. The diagonal matrix of eigenvalues $C_y$ is equivalent of the singular values square. 
3. The left singular vectors, U, turn out to be the eigenvectors of the correlation matrix, which has eigenvalues identical to those of the covariance matrix. 

In [311]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, svd_solver='arpack')
pca.fit(matrix)
comp = pca.fit_transform(matrix)

Look at the covariance Matrix

In [312]:
pca.get_covariance()

array([[   7.        ,    7.        ,    5.        ,   65.        ],
       [   7.        ,    7.        ,    5.        ,   65.        ],
       [   5.        ,    5.        ,    4.        ,   44.        ],
       [  65.        ,   65.        ,   44.        ,  617.33333333]])

From the matrix we can get it as:

In [315]:
np.dot((matrix - mean_vec).T , (matrix - mean_vec))/(matrix.shape[0]-1)

array([[   7.        ,    7.        ,    5.        ,   65.        ],
       [   7.        ,    7.        ,    5.        ,   65.        ],
       [   5.        ,    5.        ,    4.        ,   44.        ],
       [  65.        ,   65.        ,   44.        ,  617.33333333]])

Not sure why .T

In [317]:
np.cov(matrix.T)

array([[   7.        ,    7.        ,    5.        ,   65.        ],
       [   7.        ,    7.        ,    5.        ,   65.        ],
       [   5.        ,    5.        ,    4.        ,   44.        ],
       [  65.        ,   65.        ,   44.        ,  617.33333333]])

In [300]:
mean_vec

array([  3.        ,   4.        ,   7.        ,  21.33333333])

array([[   7.        ,    7.        ,    5.        ,   65.        ],
       [   7.        ,    7.        ,    5.        ,   65.        ],
       [   5.        ,    5.        ,    4.        ,   44.        ],
       [  65.        ,   65.        ,   44.        ,  617.33333333]])

In [ ]:
http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html